In [268]:
import pandas as pd
import numpy as np
import pylab as pl
import os 
import sys
import geopandas as gpd
from geopandas import GeoDataFrame
from shapely.geometry import Point
from matplotlib.pylab import plt
import statsmodels.api as sm
import seaborn as sb
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

%matplotlib inline

In [299]:
january_15 = pd.read_csv("Taxi_201501_F.csv")
fifteen = pd.read_csv("Taxi_2015_F(1).csv")

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [300]:
def weekendfilter(record):
    if record == 5 or record == 6:
        return 1
    else:
        return 0
def rush_hourfilter(record):
    if record in [7.0,8.0,9.0,16.0,17.0,18.0]:
        return 1
    else:
        return 0

In [301]:
january_15.date = pd.to_datetime(january_15.date)
fifteen.date = pd.to_datetime(fifteen.date)

january_15["Weekend"] = january_15.date.dt.dayofweek.apply(weekendfilter)
fifteen["Weekend"] = fifteen.date.dt.dayofweek.apply(weekendfilter)

january_15["Rush_Hour"] = january_15.hour.apply(rush_hourfilter)
fifteen["Rush_Hour"] = fifteen.hour.apply(rush_hourfilter)

january_15.dropna(subset = ["HOURLYDRYBULBTEMPF", "HOURLYWindSpeed", "HOURLYPrecip"], inplace=True)
fifteen.dropna(subset = ["HOURLYDRYBULBTEMPF_x", "HOURLYWindSpeed_x", "HOURLYPrecip_x"], inplace=True)

In [307]:
#fifteen.HOURLYPrecip_x.replace({"T": 0, '0.16s': .16, '0.04s': .04, '0.11s': .11, '0.60s': .6, '0.09s': .09,\
#                        '0.03s': .03, '0.02s':.02, '0.06s':.06, '0.01s': .01, '0.05s': .05}, inplace=True)

#january_15.HOURLYPrecip.replace({"T": 0, '0.16s': .16, '0.04s': .04, '0.11s': .11, '0.60s': .6, '0.09s': .09,\
#                        '0.03s': .03, '0.02s':.02, '0.06s':.06, '0.01s': .01, '0.05s': .05}, inplace=True)

#january_15.HOURLYDRYBULBTEMPF.replace({"54s": 54}, inplace=True)
#fifteen.HOURLYDRYBULBTEMPF_x.replace({"54s": 54}, inplace=True)

january_15.HOURLYPrecip = january_15.HOURLYPrecip.astype(float)
fifteen.HOURLYPrecip_x = fifteen.HOURLYPrecip_x.astype(float)


january_15.HOURLYDRYBULBTEMPF = january_15.HOURLYDRYBULBTEMPF.astype(float)
fifteen.HOURLYDRYBULBTEMPF_x = fifteen.HOURLYDRYBULBTEMPF_x.astype(float)

jan_mean = january_15.HOURLYDRYBULBTEMPF.mean()
jan_std = january_15.HOURLYDRYBULBTEMPF.std()

mean = fifteen.HOURLYDRYBULBTEMPF_x.mean()
std = fifteen.HOURLYDRYBULBTEMPF_x.std()


def jan_stdfilter(record):
    if record < jan_mean - jan_std:
        return 1
    elif record > jan_mean + jan_std:
        return 3
    else:
        return 2
    
def stdfilter(record):
    if record < mean - std:
        return 1
    elif record > mean + std:
        return 3
    else:
        return 2

In [328]:
X_int_15.isnull().sum()

2.0                       0
3.0                       0
4.0                       0
5.0                       0
6.0                       0
7.0                       0
8.0                       0
9.0                       0
10.0                      0
11.0                      0
12.0                      0
13.0                      0
14.0                      0
15.0                      0
16.0                      0
17.0                      0
18.0                      0
19.0                      0
20.0                      0
21.0                      0
22.0                      0
23.0                      0
24.0                      0
25.0                      0
26.0                      0
27.0                      0
28.0                      0
29.0                      0
30.0                      0
31.0                      0
                      ...  
240.0                     0
241.0                     0
242.0                     0
243.0                     0
244.0               

In [309]:
january_15["TempBuckets"] = january_15.HOURLYDRYBULBTEMPF.apply(jan_stdfilter)
fifteen["TempBuckets"] = fifteen.HOURLYDRYBULBTEMPF_x.apply(stdfilter)


january_15['DoW'] = january_15.date.dt.dayofweek
fifteen['DoW'] = fifteen.date.dt.dayofweek

In [330]:
y_jan = january_15.COUNTS
y_15 = fifteen.COUNTS

X_int_jan = pd.concat((pd.get_dummies(january_15.LocationID).ix[:,2:], january_15.Weekend, pd.get_dummies(january_15.TempBuckets).ix[:,2:], \
               january_15.Rush_Hour, january_15.HOURLYPrecip, january_15.HOURLYWindSpeed), axis=1)
X_nonint_jan = pd.concat((pd.get_dummies(january_15.LocationID).ix[:,2:], pd.get_dummies(january_15.DoW).ix[:,1:], pd.get_dummies(january_15.TempBuckets).ix[:,2:], \
               pd.get_dummies(january_15.hour).ix[:,1:], january_15.HOURLYPrecip, january_15.HOURLYWindSpeed), axis=1)

X_int_15 = pd.concat((pd.get_dummies(fifteen.LocationID).ix[:,2:], fifteen.Weekend, pd.get_dummies(fifteen.TempBuckets).ix[:,2:], \
               fifteen.Rush_Hour, fifteen.HOURLYPrecip_x.fillna(0), fifteen.HOURLYWindSpeed_x.fillna(0)), axis=1)
X_nonint_15 = pd.concat((pd.get_dummies(fifteen.LocationID).ix[:,2:], pd.get_dummies(fifteen.DoW).ix[:,1:], pd.get_dummies(fifteen.TempBuckets).ix[:,2:], \
               pd.get_dummies(fifteen.hour).ix[:,1:], fifteen.HOURLYPrecip_x.fillna(0), fifteen.HOURLYWindSpeed_x.fillna(0)), axis=1)

In [331]:
mv_int_jan = sm.OLS(y_jan, X_int_jan).fit()
mv_nonint_jan = sm.OLS(y_jan, X_nonint_jan).fit()

In [332]:
mv_int_15 = sm.OLS(y_15, X_int_15).fit()
mv_nonint_15 = sm.OLS(y_15, X_nonint_15).fit()

In [333]:
mv_int_jan.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 COUNTS   R-squared:                       0.508
Model:                            OLS   Adj. R-squared:                  0.507
Method:                 Least Squares   F-statistic:                     511.3
Date:                Thu, 07 Dec 2017   Prob (F-statistic):               0.00
Time:                        23:38:58   Log-Likelihood:            -9.1964e+05
No. Observations:              131551   AIC:                         1.840e+06
Df Residuals:                  131286   BIC:                         1.842e+06
Df Model:                         265                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
2.0               -40.7517     26.429     -1.542      0.123       -92.551    11.048
3.0               -38.1405     12.433     -3.068      0.002       -62.508   -13.773
4.0                64.3593     10.533      6.111      0.000        43.716    85.003
5.0               -25.0625     79.382     -0.316      0.752      -180.649   130.524
6.0               -39.3722     25.319     -1.555      0.120       -88.997    10.253
7.0               164.7054     10.509     15.672      0.000       144.107   185.303
8.0               -41.5429     18.708     -2.221      0.026       -78.211    -4.875
9.0               -39.0539     16.678     -2.342      0.019       -71.742    -6.366
10.0              -34.8377     11.539     -3.019      0.003       -57.454   -12.222
11.0              -38.0467     14.845     -2.563      0.010       -67.144    -8.950
12.0              -30.9621     11.634     -2.661      0.008       -53.765    -8.159
13.0              175.5445     10.524     16.680      0.000       154.917   196.172
14.0              -27.5740     10.635     -2.593      0.010       -48.418    -6.730
15.0              -41.4770     16.016     -2.590      0.010       -72.869   -10.085
16.0              -35.7185     11.966     -2.985      0.003       -59.171   -12.266
17.0               32.0346     10.540      3.039      0.002        11.377    52.693
18.0              -27.5674     10.668     -2.584      0.010       -48.477    -6.658
19.0              -40.8456     15.610     -2.617      0.009       -71.441   -10.251
20.0              -29.0483     10.699     -2.715      0.007       -50.018    -8.079
21.0              -38.7472     12.529     -3.093      0.002       -63.303   -14.191
22.0              -36.2236     11.663     -3.106      0.002       -59.083   -13.364
23.0              -40.9937     24.764     -1.655      0.098       -89.530     7.543
24.0               38.9267     10.540      3.693      0.000        18.269    59.584
25.0               60.0643     10.540      5.699      0.000        39.407    80.722
26.0              -33.8904     11.344     -2.988      0.003       -56.123   -11.657
27.0              -52.4388     60.426     -0.868      0.385      -170.872    65.994
28.0              -32.4698     10.772     -3.014      0.003       -53.583   -11.356
29.0              -35.2161     12.559     -2.804      0.005       -59.832   -10.600
30.0              -31.9268     79.376     -0.402      0.688      -187.503   123.649
31.0              -40.2356     13.353     -3.013      0.003       -66.407   -14.064
32.0              -35.4875     11.579     -3.065      0.002       -58.182   -12.793
33.0               59.1446     10.548      5.607      0.000        38.471    79.819
34.0              -33.9558     11.674     -2.909      0.004       -56.837   -11.075
35.0              -34.6492     11.298     -3.067      0.002

In [334]:
mv_nonint_jan.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 COUNTS   R-squared:                       0.529
Model:                            OLS   Adj. R-squared:                  0.528
Method:                 Least Squares   F-statistic:                     504.9
Date:                Thu, 07 Dec 2017   Prob (F-statistic):               0.00
Time:                        23:39:01   Log-Likelihood:            -9.1675e+05
No. Observations:              131551   AIC:                         1.834e+06
Df Residuals:                  131259   BIC:                         1.837e+06
Df Model:                         292                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [95.0% Conf. Int.]
-----------------------------------------------------------------------------------
2.0                -0.3780     26.151     -0.014      0.988       -51.633    50.877
3.0                18.4806     12.782      1.446      0.148        -6.571    43.532
4.0               129.7932     11.030     11.767      0.000       108.174   151.413
5.0                48.4062     77.789      0.622      0.534      -104.060   200.872
6.0                13.5356     25.087      0.540      0.590       -35.634    62.706
7.0               230.5538     11.010     20.940      0.000       208.974   252.134
8.0                14.0571     18.736      0.750      0.453       -22.665    50.779
9.0                14.5587     16.787      0.867      0.386       -18.343    47.460
10.0               19.7804     11.950      1.655      0.098        -3.641    43.202
11.0               15.9764     15.039      1.062      0.288       -13.499    45.452
12.0               22.8947     12.044      1.901      0.057        -0.711    46.501
13.0              241.2044     11.024     21.879      0.000       219.597   262.812
14.0               36.6080     11.123      3.291      0.001        14.808    58.408
15.0                6.9124     16.165      0.428      0.669       -24.770    38.595
16.0               25.7190     12.345      2.083      0.037         1.522    49.916
17.0               97.4207     11.038      8.826      0.000        75.786   119.055
18.0               35.4105     11.152      3.175      0.001        13.552    57.269
19.0               17.7140     15.775      1.123      0.261       -13.205    48.633
20.0               33.6138     11.181      3.006      0.003        11.700    55.527
21.0               17.6119     12.865      1.369      0.171        -7.604    42.828
22.0               21.2151     12.067      1.758      0.079        -2.436    44.866
23.0                8.4556     24.570      0.344      0.731       -39.700    56.611
24.0              104.4420     11.039      9.462      0.000        82.807   126.077
25.0              125.5796     11.039     11.376      0.000       103.944   147.215
26.0               25.0779     11.769      2.131      0.033         2.011    48.145
27.0               18.6535     59.265      0.315      0.753       -97.505   134.812
28.0               30.0691     11.248      2.673      0.008         8.023    52.115
29.0               18.0703     12.900      1.401      0.161        -7.214    43.355
30.0               22.2390     77.769      0.286      0.775      -130.187   174.665
31.0               13.6265     13.645      0.999      0.318       -13.118    40.371
32.0               18.7399     11.989      1.563      0.118        -4.759    42.239
33.0              124.4530     11.045     11.268      0.000       102.806   146.100
34.0               17.1823     12.078      1.423      0.155        -6.491    40.856
35.0               25.0407     11.730      2.135      0.033

In [335]:
mv_int_15.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 COUNTS   R-squared:                       0.550
Model:                            OLS   Adj. R-squared:                  0.550
Method:                 Least Squares   F-statistic:                     8794.
Date:                Thu, 07 Dec 2017   Prob (F-statistic):               0.00
Time:                        23:39:03   Log-Likelihood:            -1.3114e+07
No. Observations:             1905580   AIC:                         2.623e+07
Df Residuals:                 1905315   BIC:                         2.623e+07
Df Model:                         265                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------
2.0                 -16.8261      6.474     -2.599      0.009       -29.514    -4.138
3.0                 -12.3429      2.776     -4.446      0.000       -17.784    -6.901
4.0                  88.1339      2.570     34.287      0.000        83.096    93.172
5.0                 -19.6263     10.153     -1.933      0.053       -39.526     0.273
6.0                 -16.3116      4.443     -3.671      0.000       -25.019    -7.604
7.0                 195.4964      2.570     76.076      0.000       190.460   200.533
8.0                 -17.5301      4.278     -4.098      0.000       -25.915    -9.145
9.0                 -14.5844      3.079     -4.737      0.000       -20.619    -8.550
10.0                 -8.7869      2.643     -3.324      0.001       -13.968    -3.606
11.0                -14.1976      2.931     -4.844      0.000       -19.942    -8.453
12.0                 -6.9308      2.797     -2.478      0.013       -12.412    -1.449
13.0                229.9442      2.570     89.466      0.000       224.907   234.982
14.0                  5.6699      2.578      2.199      0.028         0.617    10.722
15.0                -15.4078      3.156     -4.881      0.000       -21.594    -9.221
16.0                -10.3321      2.672     -3.867      0.000       -15.569    -5.095
17.0                 77.3214      2.570     30.080      0.000        72.283    82.359
18.0                 -3.0519      2.587     -1.179      0.238        -8.123     2.019
19.0                -15.5278      3.145     -4.937      0.000       -21.692    -9.364
20.0                 -4.1353      2.593     -1.595      0.111        -9.218     0.947
21.0                -10.9056      2.688     -4.057      0.000       -16.174    -5.637
22.0                 -8.1102      2.645     -3.066      0.002       -13.294    -2.926
23.0                -15.5192      3.718     -4.174      0.000       -22.806    -8.232
24.0                 64.3998      2.571     25.052      0.000        59.361    69.438
25.0                 95.4229      2.571     37.120      0.000        90.385   100.461
26.0                 -5.6659      2.626     -2.157      0.031       -10.813    -0.518
27.0                -15.4674      8.414     -1.838      0.066       -31.958     1.024
28.0                 -6.7930      2.592     -2.621      0.009       -11.874    -1.713
29.0                -12.4878      2.789     -4.477      0.000       -17.955    -7.021
30.0                -14.6384     12.838     -1.140      0.254       -39.801    10.524
31.0                -14.4155      3.231     -4.461      0.000       -20.748    -8.083
32.0                -12.3659      2.704     -4.573      0.000       -17.666    -7.066
33.0                 93.9801      2.571     36.553      0.000        88.941    99.019
34.0                 -8.7047      2.745     -3.172      0.002       -14.0

In [336]:
mv_nonint_15.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 COUNTS   R-squared:                       0.570
Model:                            OLS   Adj. R-squared:                  0.570
Method:                 Least Squares   F-statistic:                     8662.
Date:                Thu, 07 Dec 2017   Prob (F-statistic):               0.00
Time:                        23:39:04   Log-Likelihood:            -1.3070e+07
No. Observations:             1905580   AIC:                         2.614e+07
Df Residuals:                 1905288   BIC:                         2.614e+07
Df Model:                         292                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------
2.0                 -33.7836      6.382     -5.294      0.000       -46.292   -21.275
3.0                 -19.8404      2.838     -6.991      0.000       -25.403   -14.278
4.0                  84.9181      2.647     32.086      0.000        79.731    90.105
5.0                 -39.8362      9.958     -4.001      0.000       -59.353   -20.320
6.0                 -31.8912      4.422     -7.212      0.000       -40.558   -23.225
7.0                 192.3168      2.646     72.684      0.000       187.131   197.503
8.0                 -30.4755      4.264     -7.147      0.000       -38.833   -22.118
9.0                 -24.4194      3.122     -7.823      0.000       -30.538   -18.301
10.0                -15.4392      2.714     -5.689      0.000       -20.758   -10.120
11.0                -24.1622      2.983     -8.100      0.000       -30.009   -18.316
12.0                -22.4701      2.857     -7.864      0.000       -28.070   -16.870
13.0                226.7478      2.646     85.683      0.000       221.561   231.935
14.0                  1.9978      2.653      0.753      0.452        -3.203     7.198
15.0                -29.2424      3.196     -9.150      0.000       -35.506   -22.979
16.0                -16.0307      2.741     -5.849      0.000       -21.402   -10.659
17.0                 74.1140      2.647     28.003      0.000        68.927    79.301
18.0                 -7.3823      2.662     -2.773      0.006       -12.600    -2.165
19.0                -25.1704      3.185     -7.904      0.000       -31.412   -18.929
20.0                 -8.7624      2.667     -3.285      0.001       -13.990    -3.534
21.0                -17.7238      2.755     -6.433      0.000       -23.124   -12.324
22.0                -14.2040      2.715     -5.231      0.000       -19.526    -8.882
23.0                -29.3986      3.728     -7.886      0.000       -36.705   -22.092
24.0                 61.1886      2.647     23.118      0.000        56.001    66.376
25.0                 92.2117      2.647     34.840      0.000        87.024    97.399
26.0                -11.4847      2.698     -4.257      0.000       -16.773    -6.197
27.0                -26.0605      8.266     -3.153      0.002       -42.262    -9.859
28.0                -11.1129      2.667     -4.168      0.000       -16.339    -5.887
29.0                -24.1410      2.849     -8.473      0.000       -29.726   -18.556
30.0                -21.3661     12.574     -1.699      0.089       -46.010     3.278
31.0                -30.9073      3.267     -9.462      0.000       -37.310   -24.505
32.0                -20.8227      2.770     -7.517      0.000       -26.252   -15.393
33.0                 90.7352      2.647     34.277      0.000        85.547    95.923
34.0                -21.6027      2.808     -7.693      0.000       -27.1

## To Do:

Intuitive + Lit Review models: 6 month and 12 month models

Unintuitive clustering model: 6 month, 12 month, 1 month

Jan. 15 vs. Jan. 16 Cross-validation: all 3 model types

Comparative analysis: compare the 3 model types based on cross-validation, and the 3 time series'

Ride-share modeling: 

In [157]:
from sklearn import preprocessing
from sklearn.cluster import KMeans

In [166]:
df.drop(['STATION','STATION_NAME','ELEVATION','LATITUDE','LONGITUDE',
                   'REPORTTPYE','HOURLYPRSENTWEATHERTYPE',
                   'HOURLYDRYBULBTEMPC','HOURLYWETBULBTEMPC',
                   'HOURLYDewPointTempF','HOURLYDewPointTempC',
                   'HOURLYRelativeHumidity','HOURLYWindDirection',
                   'HOURLYStationPressure','HOURLYPressureTendency',
                   'HOURLYPressureChange','HOURLYSeaLevelPressure',
                   'HOURLYAltimeterSetting','DAILYMaximumDryBulbTemp',
                   'DAILYMinimumDryBulbTemp','DAILYAverageDryBulbTemp',
                   'DAILYDeptFromNormalAverageTemp',
                   'DAILYAverageRelativeHumidity',
                   'DAILYAverageDewPointTemp','DAILYAverageWetBulbTemp',
                   'DAILYHeatingDegreeDays','DAILYCoolingDegreeDays',
                   'DAILYSunrise','DAILYSunset','DAILYWeather',
                   'DAILYPrecip','DAILYAverageStationPressure',
                   'DAILYAverageSeaLevelPressure','DAILYAverageWindSpeed',
                   'DAILYPeakWindSpeed','PeakWindDirection',
                   'DAILYSustainedWindSpeed','DAILYSustainedWindDirection',
                   'MonthlyMaximumTemp','MonthlyMinimumTemp','MonthlyMeanTemp',
                   'MonthlyAverageRH', 'MonthlyDewpointTemp',
                   'MonthlyWetBulbTemp','MonthlyAvgHeatingDegreeDays',
                   'MonthlyAvgCoolingDegreeDays','MonthlyStationPressure',
                   'MonthlySeaLevelPressure','MonthlyAverageWindSpeed',
                   'MonthlyTotalSnowfall','MonthlyDeptFromNormalMaximumTemp',
                   'MonthlyDeptFromNormalMinimumTemp',
                   'MonthlyDeptFromNormalAverageTemp',
                   'MonthlyDeptFromNormalPrecip','MonthlyTotalLiquidPrecip',
                   'MonthlyGreatestPrecip','MonthlyGreatestPrecipDate',
                   'MonthlyGreatestSnowfall','MonthlyGreatestSnowfallDate',
                   'MonthlyGreatestSnowDepth','MonthlyGreatestSnowDepthDate',
                   'MonthlyDaysWithGT90Temp','MonthlyDaysWithLT32Temp',
                   'MonthlyDaysWithGT32Temp','MonthlyDaysWithLT0Temp',
                   'MonthlyDaysWithGT001Precip','MonthlyDaysWithGT010Precip',
                   'MonthlyDaysWithGT1Snow','MonthlyMaxSeaLevelPressureValue',
                   'MonthlyMaxSeaLevelPressureDate',
                   'MonthlyMaxSeaLevelPressureTime',
                   'MonthlyMinSeaLevelPressureValue',
                   'MonthlyMinSeaLevelPressureDate',
                   'MonthlyMinSeaLevelPressureTime',
                   'MonthlyTotalHeatingDegreeDays',
                   'MonthlyTotalCoolingDegreeDays',
                   'MonthlyDeptFromNormalHeatingDD',
                   'MonthlyDeptFromNormalCoolingDD',
                   'MonthlyTotalSeasonToDateHeatingDD',
                   'MonthlyTotalSeasonToDateCoolingDD'],axis=1,inplace=True)

In [281]:
X_new_int = pd.DataFrame(preprocessing.normalize(X_int))

In [282]:
def elbow(data,K):
#data is your input as numpy form
#K is a list of number of clusters you would like to show.
    # Run the KMeans model and save all the results for each number of clusters
    KM = [KMeans(n_clusters=k).fit(data) for k in K]
    
    # Save the centroids for each model with a increasing k
    centroids = [k.cluster_centers_ for k in KM]

    # For each k, get the distance between the data with each center. 
    D_k = [cdist(data, cent, 'euclidean') for cent in centroids]
    
    # But we only need the distance to the nearest centroid since we only calculate dist(x,ci) for its own cluster.
    globals()['dist'] = [np.min(D,axis=1) for D in D_k]
    
    # Calculate the Average SSE.
    avgWithinSS = [sum(d)/data.shape[0] for d in dist]
    
    
    # elbow curve
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(K, avgWithinSS, 'b*-')
    plt.grid(True)
    plt.xlabel('Number of clusters')
    plt.ylabel('Average within-cluster sum of squares')
    plt.title('Elbow for KMeans clustering')
    plt.show()
    
    
    # Total with-in sum of square plot. Another way to show the result.
    wcss = [sum(d**2) for d in dist]
    tss = sum(pdist(data)**2)/data.shape[0]
    bss = tss-wcss
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(K, bss/tss*100, 'b*-')
    plt.grid(True)
    plt.xlabel('Number of clusters')
    plt.ylabel('Percentage of variance explained')
    plt.title('Elbow for KMeans clustering')
    plt.show()

In [298]:
elbow(X_new_int,[1,2,3,4,5,6,7,8])

KeyboardInterrupt: 

In [295]:
clusterer = KMeans(n_clusters=20, random_state=324)
X_new_int['cluster_label'] = clusterer.fit_predict(X_new_int)

In [297]:
mv = sm.OLS(list(y),pd.get_dummies(X_new_int.cluster_label).ix[:,1:]).fit()
mv.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.148
Model:                            OLS   Adj. R-squared:                  0.148
Method:                 Least Squares   F-statistic:                     1204.
Date:                Thu, 07 Dec 2017   Prob (F-statistic):               0.00
Time:                        23:11:14   Log-Likelihood:            -9.5572e+05
No. Observations:              131551   AIC:                         1.911e+06
Df Residuals:                  131532   BIC:                         1.912e+06
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
1            157.1805      3.599     43.672      0.000       150.126   164.235
2            153.6532      2.923     52.576      0.000       147.925   159.381
3            222.0397      6.662     33.330      0.000       208.982   235.097
4            160.5190      6.253     25.670      0.000       148.263   172.775
5            283.9597     13.881     20.457      0.000       256.754   311.166
6            184.4113      4.125     44.709      0.000       176.327   192.496
7             93.3015     17.125      5.448      0.000        59.737   126.866
8            118.2017     14.122      8.370      0.000        90.524   145.880
9            211.6111      4.439     47.666      0.000       202.910   220.312
10            87.1243     25.432      3.426      0.001        37.279   136.970
11           189.2760      3.088     61.292      0.000       183.223   195.329
12             8.4667     89.313      0.095      0.924      -166.585   183.518
13           111.3289      2.150     51.792      0.000       107.116   115.542
14           248.0492     11.078     22.391      0.000       226.337   269.762
15           631.0556     81.531      7.740      0.000       471.256   790.855
16           107.1722      3.859     27.773      0.000        99.609   114.736
17           109.3805      1.904     57.446      0.000       105.649   113.112
18           149.7555     12.065     12.413      0.000       126.109   173.402
19           284.4439     13.948     20.393      0.000       257.105   311.782
==============================================================================
Omnibus:                   160262.020   Durbin-Watson:                   1.301
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         33077649.397
Skew:                           6.468   Prob(JB):                         0.00
Kurtosis:                      79.598   Cond. No.                         46.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Finish 12 month / 1 month: lit review + intuitive

Uber version of the above models

Plots/numbers/stats and some analysis